# Modeling and solving search problems

Consider the following river crossing puzzle:

## Jealous Husbands Problem with an Island

There are n couples: $(𝐻_1,𝑊_1)$, $(𝐻_2,𝑊_2)$, …, $(𝐻_𝑛,𝑊_𝑛)$  where $𝐻_𝑖$ is husband $𝑖$ and $𝑊_𝑖$ is wife $𝑖$.

They need to cross from the left bank of a river to the right bank.

There is a boat that can carry at most two people at a time.

Important: There is an island midway between the left and right banks. The boat can stop at the island.

Rules:

1. Jealousy constraint: No wife may ever be in the presence of another husband (on any shore or the island or in the boat) unless her own husband is also present. Otherwise, fights (and failures) occur.

2. The boat needs at least one person to operate (no remote-controlled trips).

3. The boat can travel between:

            Left bank ↔ Island

            Island ↔ Right bank

4. All people, including the boat, start at the left bank.

Goal: get everyone (husbands and wives) safely to the right bank, obeying all the rules.


## Tasks:

1. Put the problem in a form of a search tree: root, nodes, solution and rules for constructing the childrens.
2. Solve it usinh a classic DFS
3. Solve it using a classic Back-tracking.
4. Solve it using the A* method.  

In [44]:
from itertools import combinations
import heapq

# Number of couples
n = 3

# People identifiers
husbands = [f'H{i+1}' for i in range(n)]
wives = [f'W{i+1}' for i in range(n)]
people = husbands + wives

# Initial and goal states
initial_state = (frozenset(people), frozenset(), frozenset(), 'L')  # (Left, Island, Right, BoatLoc)
goal_state = (frozenset(), frozenset(), frozenset(people), 'R')

# Constraint check
def is_valid(group):
    for i in range(n):
        w = f'W{i+1}'
        h = f'H{i+1}'
        if w in group and h not in group:
            for j in range(n):
                if j != i and f'H{j+1}' in group:
                    return False
    return True

# Full state validity
def is_valid_state(left, island, right):
    return all(is_valid(loc) for loc in [left, island, right])

# Move logic
def move_people(state, movers):
    left, island, right, boat_loc = state
    left, island, right = set(left), set(island), set(right)
    movers = set(movers)

    if boat_loc == 'L':
        if movers <= left:
            left -= movers
            island |= movers
            new_state = (frozenset(left), frozenset(island), frozenset(right), 'I')
    elif boat_loc == 'I':
        if movers <= island:
            island -= movers
            # Try both directions: to Left or Right
            states = []
            left1 = frozenset(left | movers)
            if is_valid_state(left1, frozenset(island), frozenset(right)):
                states.append((frozenset(left | movers), frozenset(island), frozenset(right), 'L'))
            right1 = frozenset(right | movers)
            if is_valid_state(frozenset(left), frozenset(island), right1):
                states.append((frozenset(left), frozenset(island), frozenset(right | movers), 'R'))
            return states
        else:
            return []
    elif boat_loc == 'R':
        if movers <= right:
            right -= movers
            island |= movers
            new_state = (frozenset(left), frozenset(island), frozenset(right), 'I')
    else:
        return []

    if is_valid_state(left, island, right):
        return [(frozenset(left), frozenset(island), frozenset(right), new_state[3])]
    return []

# Generate all valid children states
def get_children(state):
    left, island, right, boat_loc = state
    location = {'L': left, 'I': island, 'R': right}[boat_loc]
    children = []
    for n_ppl in [1, 2]:
        for movers in combinations(location, n_ppl):
            results = move_people(state, movers)
            for new_state in results:
                children.append(new_state)
    return children

# DFS  
def dfs(state):
    # Initialize the stack with the current state and its path
    stack = [(state, [state])]
    
    # Set to keep track of visited states
    visited = set()
    
    while stack:
        # Pop the most recent state and path from the stack
        current_s, current_path = stack.pop()
        if current_s in visited:
            continue
        visited.add(current_s)
        
        # If the goal state is reached, return the current path
        if current_s == goal_state:
            return current_path
        for child in get_children(current_s):
            if child not in visited:
                stack.append((child, current_path + [child]))
    
    # If no solution found, return None
    return None



# Backtracking
def backtracking(state, visited=None, path=None):
    if visited is None: visited = set()
    if path is None: path = []

    if state == goal_state:
        return path + [state]

    visited.add(state)
    for child in get_children(state):
        if child not in visited:
            result = backtracking(child, visited, path + [state])
            if result: return result
    return None

# Heuristic for A* (number of people not at right bank)
def heuristic(state):
    left, island, right, _ = state
    return len(left) + len(island)

# A* Search
def a_star(start_state):
    heap = []
    heapq.heappush(heap, (heuristic(start_state), 0, start_state, []))
    visited = set()

    while heap:
        _, cost, current, path = heapq.heappop(heap)
        if current in visited: continue
        visited.add(current)

        if current == goal_state:
            return path + [current]

        for child in get_children(current):
            if child not in visited:
                heapq.heappush(heap, (cost + 1 + heuristic(child), cost + 1, child, path + [current]))
    return None

# Run the algorithms
dfs_path = dfs(initial_state)
backtracking_path = backtracking(initial_state)
astar_path = a_star(initial_state)

# To display the actual steps:
def print_path(path):
    for i, state in enumerate(path):
        L, I, R, B = state
        print(f"Step {i}:")
        print(f"  Left: {sorted(L)}")
        print(f"  Island: {sorted(I)}")
        print(f"  Right: {sorted(R)}")
        print(f"  Boat at: {B}\n")

# Print results

In [45]:
print("DFS path length:", len(dfs_path))
print_path(dfs_path)

DFS path length: 129
Step 0:
  Left: ['H1', 'H2', 'H3', 'W1', 'W2', 'W3']
  Island: []
  Right: []
  Boat at: L

Step 1:
  Left: ['H1', 'H3', 'W1', 'W3']
  Island: ['H2', 'W2']
  Right: []
  Boat at: I

Step 2:
  Left: ['H1', 'H3', 'W1', 'W3']
  Island: []
  Right: ['H2', 'W2']
  Boat at: R

Step 3:
  Left: ['H1', 'H3', 'W1', 'W3']
  Island: ['H2']
  Right: ['W2']
  Boat at: I

Step 4:
  Left: ['H1', 'H2', 'H3', 'W1', 'W3']
  Island: []
  Right: ['W2']
  Boat at: L

Step 5:
  Left: ['H2', 'H3', 'W3']
  Island: ['H1', 'W1']
  Right: ['W2']
  Boat at: I

Step 6:
  Left: ['H1', 'H2', 'H3', 'W3']
  Island: ['W1']
  Right: ['W2']
  Boat at: L

Step 7:
  Left: ['H3', 'W3']
  Island: ['H1', 'H2', 'W1']
  Right: ['W2']
  Boat at: I

Step 8:
  Left: ['H3', 'W3']
  Island: ['W1']
  Right: ['H1', 'H2', 'W2']
  Boat at: R

Step 9:
  Left: ['H3', 'W3']
  Island: ['W1', 'W2']
  Right: ['H1', 'H2']
  Boat at: I

Step 10:
  Left: ['H3', 'W3']
  Island: []
  Right: ['H1', 'H2', 'W1', 'W2']
  Boat at: R

In [46]:
print("Backtracking path length:", len(backtracking_path))
print_path(backtracking_path)

Backtracking path length: 167
Step 0:
  Left: ['H1', 'H2', 'H3', 'W1', 'W2', 'W3']
  Island: []
  Right: []
  Boat at: L

Step 1:
  Left: ['H1', 'H2', 'H3', 'W2']
  Island: ['W1', 'W3']
  Right: []
  Boat at: I

Step 2:
  Left: ['H1', 'H2', 'H3', 'W2', 'W3']
  Island: ['W1']
  Right: []
  Boat at: L

Step 3:
  Left: ['H1', 'H2', 'H3', 'W3']
  Island: ['W1', 'W2']
  Right: []
  Boat at: I

Step 4:
  Left: ['H1', 'H2', 'H3', 'W1', 'W3']
  Island: ['W2']
  Right: []
  Boat at: L

Step 5:
  Left: ['H1', 'H2', 'H3', 'W1']
  Island: ['W2', 'W3']
  Right: []
  Boat at: I

Step 6:
  Left: ['H1', 'H2', 'H3', 'W1', 'W2']
  Island: ['W3']
  Right: []
  Boat at: L

Step 7:
  Left: ['H1', 'H2', 'W1', 'W2']
  Island: ['H3', 'W3']
  Right: []
  Boat at: I

Step 8:
  Left: ['H1', 'H2', 'W1', 'W2']
  Island: []
  Right: ['H3', 'W3']
  Boat at: R

Step 9:
  Left: ['H1', 'H2', 'W1', 'W2']
  Island: ['H3']
  Right: ['W3']
  Boat at: I

Step 10:
  Left: ['H1', 'H2', 'H3', 'W1', 'W2']
  Island: []
  Right: 

In [47]:
print("A* path length:", len(astar_path))
print_path(astar_path)

A* path length: 19
Step 0:
  Left: ['H1', 'H2', 'H3', 'W1', 'W2', 'W3']
  Island: []
  Right: []
  Boat at: L

Step 1:
  Left: ['H1', 'H2', 'H3', 'W1']
  Island: ['W2', 'W3']
  Right: []
  Boat at: I

Step 2:
  Left: ['H1', 'H2', 'H3', 'W1']
  Island: []
  Right: ['W2', 'W3']
  Boat at: R

Step 3:
  Left: ['H1', 'H2', 'H3', 'W1']
  Island: ['W3']
  Right: ['W2']
  Boat at: I

Step 4:
  Left: ['H1', 'H2', 'H3', 'W1', 'W3']
  Island: []
  Right: ['W2']
  Boat at: L

Step 5:
  Left: ['H1', 'H2', 'W1']
  Island: ['H3', 'W3']
  Right: ['W2']
  Boat at: I

Step 6:
  Left: ['H1', 'H2', 'H3', 'W1']
  Island: ['W3']
  Right: ['W2']
  Boat at: L

Step 7:
  Left: ['H1', 'W1']
  Island: ['H2', 'H3', 'W3']
  Right: ['W2']
  Boat at: I

Step 8:
  Left: ['H1', 'W1']
  Island: ['W3']
  Right: ['H2', 'H3', 'W2']
  Boat at: R

Step 9:
  Left: ['H1', 'W1']
  Island: ['H3', 'W3']
  Right: ['H2', 'W2']
  Boat at: I

Step 10:
  Left: ['H1', 'H3', 'W1']
  Island: ['W3']
  Right: ['H2', 'W2']
  Boat at: L

St